In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import argparse
import json
import pprint
import requests
import sys
import urllib
import json
import threading



In [2]:
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode

In [3]:
# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
MATCH_PATH = '/v3/businesses/matches'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
# Defaults for our simple example.
#DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'New York City, NY'
SEARCH_LIMIT = 10

In [4]:
print(MATCH_PATH,type(MATCH_PATH)) 
print(quote(MATCH_PATH.encode('utf8')),type(quote(MATCH_PATH.encode('utf8'))))
print(MATCH_PATH == quote(MATCH_PATH.encode('utf8')))
print(MATCH_PATH is quote(MATCH_PATH.encode('utf8')))
print(API_HOST+MATCH_PATH == '{0}{1}'.format(API_HOST, quote(MATCH_PATH.encode('utf8'))))

/v3/businesses/matches <class 'str'>
/v3/businesses/matches <class 'str'>
True
False
True


In [5]:

def query_match(term, location, api_key):
    """Query the Business Match endpoint by business name and address.
    Args:
        term (str): The business name passed to the API.
        location (str): The business location passed to the API.
        Note city and state parameters are set to default to be New York City and NY
    Returns:
        dict: The JSON response from the request.
    """
    name = '?name='+term.replace(' ','+')
    address = 'address1='+location.replace(' ','+')
    headers = {'Authorization': 'Bearer %s' % api_key,}
    url = API_HOST + MATCH_PATH + name +'&city=New+York+City&state=NY&country=US&'+ address
    return requests.request('GET',url,headers = headers).json()

In [6]:
def get_business(business_id, api_key,url_params = None):
    """Query the Business Detail endpoint by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id
    url_params = url_params or {}
    url = '{0}{1}'.format(API_HOST, quote(business_path.encode('utf8'))) # Why the path need to encode and quote
    headers = {'Authorization': 'Bearer %s' % api_key,}

    # print(u'Querying {0} ...'.format(url))
    response = requests.request('GET', url, headers=headers, params=url_params)
    return response.json()

    #return request(API_HOST, business_path, api_key)



In [7]:
def batch_process_3000(start, end, api_key):

    local_df = df.iloc[start:end+1,:]
    
    for i in range(start,end+1):
        if i % 50 == 0:
            print("Thread-{}: Querying {}th entry".format(start, i))
        try:
            response = query_match(local_df['DBA'][i], local_df['ADDRESS'][i], api_key)
            if 'error' in response and 'code' in response['error'] and response['error']['code'] == 'ACCESS_LIMIT_REACHED':
                print("Thread-{} Stopped: ACCESS LIMIT REACHED!!!".format(start))
                break
            business = response['businesses']
            if business:
                data = get_business(business[0]['id'], api_key)
                for key in data.keys():
                    if key not in keep:
                        continue
                    elif key in keep1:
                        local_df.loc[i,'yelp_'+key] = data[key]
                    elif key == 'location':
                        for j in location_keys:
                            local_df.loc[i,'yelp_'+j] = data[key][j]
                    elif key == 'coordinates':
                        local_df.loc[i,'yelp_latitude'] = data[key]['latitude']
                        local_df.loc[i,'yelp_longitude']  = data[key]['longitude']
                    elif key == 'transactions':
                        local_df.loc[i,'yelp_'+key] = (', '.join(data['transactions']))
                    elif key == 'categories':
                        temp1 = json.dumps([item['alias'] for item in data[key]])
                        temp2 = json.dumps([item['title'] for item in data[key]])
                        local_df.loc[i,'yelp_'+key+'_a'] = temp1[1:len(temp1)-1]
                        local_df.loc[i,'yelp_'+key+'_t'] = temp2[1:len(temp2)-1]
                    elif key == 'hours':
                        # dic = {item['day']:item['start']+','+item['end'] for item in data[key][0]['open']} 
                        dic = {item['day']:item['start']+item['end'] for item in data[key][0]['open']}
                        for day in range(7):
                            if day in dic.keys():
                                local_df.loc[i,'yelp_day'+str(day)] = dic[day]
                            # local_df.loc[i,'yelp_day'+str(day)] = dic[day] if day in dic.keys() else np.nan
                    elif key == 'price':
                        local_df.loc[i,'yelp_'+key] = len(data[key])

            else:
                print("Thread-{}: no business found for {}. Position-{}".format(start, local_df['DBA'][i], i))
        except json.decoder.JSONDecodeError:
            print("Thread-{}: Json Decoder Error in running {}. Position-{}".format(start, local_df['DBA'][i], i))
        
        except HTTPError as error:
            sys.exit(
                'Encountered HTTP error {0} on {1}:\n {2}\nAbort program.'.format(
                    error.code,error.url,error.read(),
                ))    
        except Exception as e:
            print("Thread-{}: Run in to error!!! Position-{}".format(start, i))
            print(e)
                
            
        
    local_df.to_csv('1001_Project_data_{}.csv'.format(start))
    return local_df

In [8]:
df = pd.read_csv('NY_Inspection.csv')
keep1 = ['name','id','is_closed','review_count','rating','phone','url']
keep = ['name','id','is_closed','review_count','rating','phone','location','coordinates','categories','price','hours','transactions','url']

location_keys = ['address1','city','state','zip_code']

API_KEYS = ['DnKa6kFX558t8UIkXIITOk5NGiUW1H4ypRVTJ6caM_bGd_ukfcs1GugjdK73JjWD_DGzN1kZuQ9ehKs9NnW6iz-zHQrlyVpTx87mro33EJ5ukYraRaw1kQ_oToPLW3Yx',
'aA1yqFtuThy9pQMwDBNXoQsIEIBhwsfagdZWeO2se7cwM_utRD8cpX2LAPkTpQV8BJhBX7-DRLQxQfJR9CSjIGOzKMPKufCF_nKSTD0SwXhVhPZ0uIxRtvcrhN7tW3Yx',
'I4EpqW3yupDMXGdxyNAUrYPStOBulGvAVSs6C7Z72T84CCV4_OdlZwfn3dLe9MHU48xSUXZeoacYy_1kxc4lZbjKuzijxb0TCAc9mIpTC8D9OSgxmIxRJG0YFBfZW3Yx',
'agUXwmM8cW5bCP2js7Iceu_YPvIxIT4tdUbPdKHL9QYxB6BbSIOJ3rPx_icxo260xzwobeOvfd4wncfZ02mhyR4mWj16hoQT1ynSEQlLH3E3brDsisnKwuhWaaj0W3Yx',
'opNQnuXzHK-dTyi5qhs9mxcs3_VbD7vZKjAnHij-n0BH8W6Y9-56SJpRXZSZ9b8KEsgWmzTUIWXCOOp5Olyh8yG1qwX-sTaJuxfFauvuYI2TUueHCGPJXDPMlar0W3Yx',
'Ey8H8bHU6_BIUVL04isi9wyox74i1YV1QSxjlMf_EbwqOycls7V8d2zJoyZtzyNZq73PIQv8qOnY1Eti_xrqCsvGyzyyFGAgNeQzuRxTRbTr_TKnpUxhdOAz5ZDrW3Yx',
'aZRnf-PXJMCCtLacE1uJ6uKRJYPyuhIbRU8Bly4NOyAMeFO7aXIKcCG2fl7Hui4CEN22BRRKDbfXhEoKrr4_JNFPXF3VSs0ajxmbOCiVqYhFXDmzZaw_v_SMm-ztW3Yx',
'tBr90NpecyROtHshn7aZtuK7GOPfRIbHnBXkX6ZCaNZ-7TFLEk0GCtqOcF0GXm6h6D17nOPay1lNiAns2oxqbKbyPuNVdLOrkQs5hRTeqRG1qsLGV58QRVJZVgrqW3Yx',
'392qu0-zBLl7823TDUDCvRSghalLly-pEPb-6-F2rROzGMk_R2MZTYyocueJQ9ZAlagGiGOOUToIStluP6Sdr9q5WIIHQ5QaM7N6CyeIcOK0FjYZG_l7EY1bCiT4W3Yx',
'XLmmmQu_kBccKnKMkzTrghXuYIdn3w13xKTMi0xrBm9ggDnNMl0Ofna64NKqJedMn34qeGZv4XoTdJzjP8piYhqsbTWYPSLyXwbxtuZaIAGkjxbLcqRbTjH-NB34W3Yx',
'vcJ9Op_XhRA4I4w2mWT2Tu98DS7wFLEbbkdbutVXwIHLZ4OuzIFz89tTt9ooZqJmuD4Jiap-yvGQe46ERb9P5UVJJBcIHi2wspqkbA0JEQj3MnV-fZc_53TJN6brW3Yx',
'6Lj-KuXW4n83sRT2G2HR22WFWDqUEQ8Y9S8K2EfygF5Dk76T8oT0oBp_tAIFRR2FY85QmE3YGA4WN4-v0sF5K9OZBWh0Ldr32a-ILd9qi9pV-BFeUqYnylK9p9j4W3Yx']


In [9]:
df.shape

(25750, 29)

In [10]:
# #threading.Thread(target=batch_process_3000, args=(132, 132, API_KEYS[0])).start() still wrong
# local_df = pd.DataFrame(df.iloc[132:133,:])
# print(local_df)
# response = query_match(local_df['DBA'][132], local_df['ADDRESS'][132], API_KEYS[0])
# business = response['businesses']
# data = get_business(business[0]['id'], API_KEYS[0])
# print(data)
# dic = {item['day']:item['start']+item['end'] for item in data['hours'][0]['open']}
# print(dic)

In [11]:
threading.Thread(target=batch_process_3000, args=(12500, 12999, API_KEYS[0])).start()
threading.Thread(target=batch_process_3000, args=(13000, 13499, API_KEYS[0])).start()
threading.Thread(target=batch_process_3000, args=(13500, 13999, API_KEYS[0])).start()
threading.Thread(target=batch_process_3000, args=(14000, 14499, API_KEYS[0])).start()
threading.Thread(target=batch_process_3000, args=(14500, 14999, API_KEYS[1])).start()
threading.Thread(target=batch_process_3000, args=(15000, 15499, API_KEYS[1])).start()
threading.Thread(target=batch_process_3000, args=(15500, 15999, API_KEYS[1])).start()
threading.Thread(target=batch_process_3000, args=(16000, 16499, API_KEYS[1])).start()
threading.Thread(target=batch_process_3000, args=(16500, 16999, API_KEYS[2])).start()
threading.Thread(target=batch_process_3000, args=(17000, 17499, API_KEYS[2])).start()
threading.Thread(target=batch_process_3000, args=(17500, 17999, API_KEYS[2])).start()
threading.Thread(target=batch_process_3000, args=(18000, 18499, API_KEYS[2])).start()
threading.Thread(target=batch_process_3000, args=(18500, 18999, API_KEYS[3])).start()
threading.Thread(target=batch_process_3000, args=(19000, 19499, API_KEYS[3])).start()
threading.Thread(target=batch_process_3000, args=(19500, 19999, API_KEYS[3])).start()

Thread-12500: Querying 12500th entry
Thread-13000: Querying 13000th entryThread-13500: Querying 13500th entry
Thread-14000: Querying 14000th entry
Thread-14500: Querying 14500th entry
Thread-15000: Querying 15000th entry

Thread-15500: Querying 15500th entry
Thread-16000: Querying 16000th entry
Thread-16500: Querying 16500th entry
Thread-17000: Querying 17000th entry
Thread-17500: Querying 17500th entry
Thread-18000: Querying 18000th entryThread-18500: Querying 18500th entry

Thread-19000: Querying 19000th entry
Thread-19500: Querying 19500th entry
Thread-13000: no business found for LUIGI'S PIZZA II. Position-13000
Thread-16000: no business found for NAIA DELI GRILL JUICE AND SALAD BAR. Position-16000
Thread-14500: no business found for AIRS CHAMPAGNE PARLOR. Position-14500


/Users/jialing/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jialing/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Thread-15000: no business found for NEW TOPAZ. Position-15000
Thread-17000: no business found for LOVE BAKERY & CAFE. Position-17000
Thread-14500: no business found for Hampton Inn JFK. Position-14501
Thread-17000: no business found for BAR GARDEN. Position-17001
Thread-17000: no business found for CRAFT BEER GARDEN. Position-17002
Thread-15500: no business found for APPLE GARDEN. Position-15501
Thread-17000: no business found for GARDEN MARKET. Position-17003
Thread-14000: no business found for SUBWAY. Position-14001
Thread-19500: no business found for BLUE BOTTLE COFFEE. Position-19501
Thread-13500: no business found for HUMBOLDT & JACKSON. Position-13501
Thread-13000: no business found for CHERRY VALLEY DELI. Position-13002
Thread-17000: no business found for BAR AT THE GARDEN. Position-17004
Thread-14000: no business found for DELI AND PIZZA. Position-14002
Thread-17000: no business found for HAMBURGER/DAILY BURGER. Position-17005
Thread-13000: no business found for LADUREE SOHO. P

Thread-18000: Run in to error!!! Position-18028
'businesses'
Thread-15000: no business found for KIMBOB NARA. Position-15025
Thread-12500: no business found for SHIRO OF JAPAN. Position-12524
Thread-12500: no business found for VOYAGE. Position-12525
Thread-13000: no business found for LA PARADA II RESTAURANT. Position-13029
Thread-17500: no business found for CHARO RESTAURANT. Position-17527
Thread-15000: no business found for CHINA WOK RESTAURANT. Position-15027
Thread-13500: no business found for LA FOGATA. Position-13527
Thread-19500: no business found for NEW PHO TAY HO 86 VIETNAMESE. Position-19529
Thread-18500: no business found for LEE CHINESE RESTAURANT. Position-18529
Thread-16500: no business found for SUGAR & CAKE FANTASIES BAKERY. Position-16527
Thread-19500: no business found for STAR 99C PIZZA. Position-19530
Thread-14500: no business found for MEZQUITE. Position-14525
Thread-12500: no business found for NEW JIN XIN RESATURANT. Position-12528
Thread-19000: no business fo

Thread-17500: Querying 17550th entry
Thread-16000: no business found for FIRST HAND FOOD PRODUCT. Position-16050Thread-14500: Querying 14550th entry

Thread-12500: no business found for FOGO DE CHAO. Position-12551
Thread-13500: no business found for TASTY DELIGHT BAKERY. Position-13551
Thread-15000: no business found for OCAFE NEW SCHOOL. Position-15052
Thread-17500: no business found for J & M BELLA PIZZA. Position-17550
Thread-16000: no business found for UNCLE LOUIE GS ITALIAN ICES & ICE CREAM. Position-16051
Thread-18000: no business found for JUDY. Position-18053
Thread-14000: no business found for TERIYAKI JAPAN. Position-14053
Thread-19000: no business found for BOGATY. Position-19048
Thread-18500: no business found for LITTLE NEGRIL JAMAICAN CUISINE. Position-18556
Thread-16000: no business found for DUSK TILL DAWN. Position-16052
Thread-14500: no business found for QED. Position-14551
Thread-15500: no business found for SAFARI. Position-15553
Thread-13000: no business found 

...

Thread-19500: Run in to error!!! Position-19919
'businesses'
Thread-17000: no business found for THE PIERPONT LEADERSHIP CENTER. Position-17444
Thread-15000: no business found for MCDONALD'S. Position-15415
Thread-13500: no business found for JULIA'S BEER AND WINE BAR. Position-13913
Thread-15500: Run in to error!!! Position-15912
'float' object has no attribute 'replace'
Thread-18000: Run in to error!!! Position-18415Thread-16500: Run in to error!!! Position-16902
'businesses'

'businesses'Thread-14000: Run in to error!!! Position-14412
'businesses'

Thread-16000: Run in to error!!! Position-16410Thread-13500: no business found for SAHARA NEW YORK. Position-13914

'businesses'
Thread-14500: Run in to error!!! Position-14911
'businesses'
Thread-13500: no business found for WAH YONG KITCHEN. Position-13915
Thread-19500: Run in to error!!! Position-19920
'businesses'
Thread-18500: Run in to error!!! Position-18918
'businesses'
Thread-12500: Run in to error!!! Position-12902
'businesses'


Thread-13500: Run in to error!!! Position-13933
'businesses'Thread-15500: Run in to error!!! Position-15932

'businesses'
Thread-14500: Run in to error!!! Position-14930Thread-12500: Run in to error!!! Position-12920
'businesses'

'businesses'
Thread-15000: Run in to error!!! Position-15434
'businesses'
Thread-16000: Run in to error!!! Position-16432
Thread-18500: Run in to error!!! Position-18936'businesses'

'businesses'
Thread-17500: no business found for SAIDY'S CAFE. Position-17939
Thread-15000: no business found for WOLFETONE'S IRISH PUB & KITCHEN. Position-15435
Thread-15500: Run in to error!!! Position-15933
'businesses'
Thread-13500: Run in to error!!! Position-13934
'businesses'
Thread-18000: Run in to error!!! Position-18436Thread-14500: Run in to error!!! Position-14931
'businesses'

'businesses'Thread-12500: Run in to error!!! Position-12921
'businesses'

Thread-19000: Run in to error!!! Position-19429
'businesses'
Thread-18500: Run in to error!!! Position-18937
'businesse

Thread-19000: no business found for NEUBERGER BERMAN EDR. Position-19447
Thread-15500: no business found for NATHAN'S FAMOUS. Position-15955
Thread-19500: no business found for COFFEE DELIGHT. Position-19966
Thread-19500: no business found for MARCHINI RESTAURANT. Position-19967
Thread-13500: no business found for PINGS. Position-13956
Thread-19000: Querying 19450th entry
Thread-13000: no business found for YOFRESH NEW YORK. Position-13455
Thread-19000: no business found for CHUNG KIWA RESTAURANT. Position-19450
Thread-15000: no business found for EUROPAN CAFE (GATE 12). Position-15456
Thread-14000: Run in to error!!! Position-14458
'businesses'
Thread-19500: Run in to error!!! Position-19969
'businesses'
Thread-16500: Run in to error!!! Position-16942
Thread-13000: no business found for HESTIA MARKETPLACE. Position-13456
'businesses'
Thread-17000: Run in to error!!! Position-17486
'businesses'
Thread-13500: no business found for FUEL FEVER. Position-13959
Thread-12500: Run in to error

Thread-18500: Run in to error!!! Position-18969
'businesses'
Thread-15000: Run in to error!!! Position-15468
'businesses'
Thread-17000: Run in to error!!! Position-17497
'businesses'
Thread-19000: Run in to error!!! Position-19462
'businesses'
Thread-15500: Run in to error!!! Position-15969
'businesses'
Thread-14500: Run in to error!!! Position-14966
'businesses'
Thread-18500: Run in to error!!! Position-18970
'businesses'
Thread-18000: no business found for THE EGAN PAVILLION. Position-18475
Thread-19500: no business found for CHAWLAS2. Position-19985
Thread-15000: Run in to error!!! Position-15469
'businesses'
Thread-17500: no business found for LE PAIN QUOTIDIEN. Position-17976
Thread-18000: no business found for DORA RESTAURANT. Position-18476
Thread-13000: no business found for ANAIAH RESTAURANT. Position-13469
Thread-12500: Run in to error!!! Position-12955
'businesses'
Thread-13500: no business found for CASA DEL CHEF BISTRO. Position-13973
Thread-16500: Run in to error!!! Posit

Thread-17500: Run in to error!!! Position-17996
'businesses'
Thread-16500: no business found for FAMOUS & FRESH 99Â¢ PIZZA. Position-16975
Thread-19000: no business found for POCHANA. Position-19480
Thread-15000: Run in to error!!! Position-15488
'businesses'
Thread-14000: no business found for M AND G RESTAURANT. Position-14491
Thread-16500: no business found for GREEN ROOM. Position-16976
Thread-14500: no business found for L' ANTAGONISTE. Position-14987
Thread-14000: no business found for NUEVO HORIZONTE ( Herbalife). Position-14492
Thread-19000: no business found for FORDE'S SPORTS BAR. Position-19482
Thread-15000: no business found for HOUSE OF ANTOJITOS. Position-15490
Thread-16500: no business found for BT3 AISLE A. Position-16978
Thread-13000: no business found for JUICE GENERATION. Position-13491
Thread-19000: no business found for MD BAGELS. Position-19483
Thread-15500: no business found for TWIN SISTER PAN. Position-15994
Thread-13500: no business found for RED LOBSTER. Posi